### Carga de la tabla results_country_prod_company en la capa Gold

#### Paso 0 - Actualizando las variables globales y las funciones comunes.

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leyendo todas las tablas necesarias."

In [0]:
# movies_df = spark.read.parquet(f"{silver_folder_path}/movies") \
#                     .filter("year_release_date >= 2010") \
#                     .filter(f"file_date = '{v_file_date}'")

movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
                        .filter("year_release_date >= 2010") \
                        .filter(f"file_date = '{v_file_date}'")

In [0]:
#movies_df.count()

In [0]:
# production_country_df = spark.read.parquet(f"{silver_folder_path}/productions_countries") \
#                                 .filter(f"file_date = '{v_file_date}'")

production_country_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_countries") \
                                .filter(f"file_date = '{v_file_date}'")

In [0]:
# country_df = spark.read.parquet(f"{silver_folder_path}/countries")
country_df = spark.read.format("delta").load(f"{silver_folder_path}/countries")

In [0]:
# movie_company_df = spark.read.parquet(f"{silver_folder_path}/movies_companies") \
#                                 .filter(f"file_date = '{v_file_date}'")

movie_company_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_companies") \
                                .filter(f"file_date = '{v_file_date}'")

In [0]:
# production_company_df = spark.read.parquet(f"{silver_folder_path}/productions_companies")
production_company_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_companies")

### Paso 2 - Generando el DataFrame con todos los requisitos

- Realiza un **inner join** entre todas las tablas  
- Selecciona todos los campos necesarios  
- Elimina todas las filas duplicadas  
- Ordena por **título de la pelicula** en orden ascendente  
- Llama a la función para agregar la **fecha y hora actual**

In [0]:
results_country_prod_company_df = movies_df.join(movie_company_df, 
                                                  movies_df.movie_id == movie_company_df.movie_id,
                                                  "inner") \
                                                .join(production_company_df, 
                                                  movie_company_df.company_id == production_company_df.company_id,
                                                  "inner") \
                                            .join(production_country_df, 
                                                  movies_df.movie_id == production_country_df.movie_id,
                                                  "inner") \
                                            .join(country_df, 
                                                  production_country_df.country_id == country_df.country_id,
                                                  "inner") \
                                            .select(movies_df.movie_id,
                                                    movie_company_df.company_id,
                                                    production_country_df.country_id,
                                                    movies_df.title,
                                                    movies_df.budget,
                                                    movies_df.revenue,
                                                    movies_df.duration_time,
                                                    movies_df.release_date,
                                                    country_df.country_name,
                                                    production_company_df.company_name) \
                                            .sort(movies_df.title.asc())

In [0]:
from pyspark.sql.functions import lit

#results_country_prod_company_final_df = add_ingestion_date(results_country_prod_company_df)
results_country_prod_company_final_df = results_country_prod_company_df \
                                        .withColumn("created_date", lit(v_file_date))

### Paso 3 - Generando el archivo final en la carpeta Gold

In [0]:
#drop_partition_if_exists(results_country_prod_company_final_df, "movie_gold", "results_country_prod_company", "created_date")

In [0]:
#results_country_prod_company_final_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_country_prod_company")

# results_country_prod_company_final_df.write.mode("append") \
#                                     .partitionBy("created_date") \
#                                     .format("parquet") \
#                                     .saveAsTable("movie_gold.results_country_prod_company")

#Caso a tabela não existir, cria a tabela com os dados, senão efetua um merge dos dados na tabela
merge_delta_lake(results_country_prod_company_final_df, "movie_gold", "results_country_prod_company", gold_folder_path, "tgt.movie_id = src.movie_id AND tgt.company_id = src.company_id AND tgt.country_id = src.country_id", "created_date")

In [0]:
#display(spark.read.parquet(f"{gold_folder_path}/results_country_prod_company"))
spark.sql("SELECT created_date, count(*) \
           FROM movie_gold.results_country_prod_company \
           GROUP BY created_date").display()